In [1]:
! pip install mlxtend

In [ ]:
! pip install xlrd

##Association Rules Generation from Frequent Itemsets
Source: https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

In [16]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#Onion, Sausages, Cheese, Water, Butter, Sugar, Eggs
df = [['Onion', 'Sausages', 'Cheese', 'Butter'],
      ['Onion', 'Sausages', 'Water', 'Sugar'],
      ['Onion', 'Water', 'Sausages'],
      ['Butter', 'Sugar', 'Eggs'],
      ['Butter', 'Sugar', 'Eggs', 'Cheese'],
      ['Water', 'Cheese', 'Eggs'],
      ['Water', 'Butter'],
      ['Onion', 'Butter', 'Sugar'],
      ['Onion', 'Butter', 'Cheese'],
      ['Onion', 'Butter', 'Water'],
      ]
#df a questo punto è già pronto come input
#lo trasform in DataFrame e lo riconverto in list per far vedere l'algoritmo
df = pd.DataFrame(df)
df

,0,1,2,3
0,Onion,Sausages,Cheese,Butter
1,Onion,Sausages,Water,Sugar
2,Onion,Water,Sausages,None
3,Butter,Sugar,Eggs,None
4,Butter,Sugar,Eggs,Cheese
5,Water,Cheese,Eggs,None
6,Water,Butter,None,None
7,Onion,Butter,Sugar,None
8,Onion,Butter,Cheese,None
9,Onion,Butter,Water,None


In [17]:
#1. conversione in list: il problema che ha None values
df = df.values.tolist()
df

[['Onion', 'Sausages', 'Cheese', 'Butter'],
 ['Onion', 'Sausages', 'Water', 'Sugar'],
 ['Onion', 'Water', 'Sausages', None],
 ['Butter', 'Sugar', 'Eggs', None],
 ['Butter', 'Sugar', 'Eggs', 'Cheese'],
 ['Water', 'Cheese', 'Eggs', None],
 ['Water', 'Butter', None, None],
 ['Onion', 'Butter', 'Sugar', None],
 ['Onion', 'Butter', 'Cheese', None],
 ['Onion', 'Butter', 'Water', None]]

In [18]:
#2. Removing None values in list, 2 dimensions
df_ = list()
for _ in df:
  #using list comprehension 
  _ = [x for x in _ if x is not None]
  df_.append(_)
df = df_
df

[['Onion', 'Sausages', 'Cheese', 'Butter'],
 ['Onion', 'Sausages', 'Water', 'Sugar'],
 ['Onion', 'Water', 'Sausages'],
 ['Butter', 'Sugar', 'Eggs'],
 ['Butter', 'Sugar', 'Eggs', 'Cheese'],
 ['Water', 'Cheese', 'Eggs'],
 ['Water', 'Butter'],
 ['Onion', 'Butter', 'Sugar'],
 ['Onion', 'Butter', 'Cheese'],
 ['Onion', 'Butter', 'Water']]

In [19]:
#df: deve essere una lista a 2 dimensioni per funzionare
#one_hot encoding (boolean output)
te = TransactionEncoder()
te_ary = te.fit(df).transform(df)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Butter,Cheese,Eggs,Onion,Sausages,Sugar,Water
0,True,True,False,True,True,False,False
1,False,False,False,True,True,True,True
2,False,False,False,True,True,False,True
3,True,False,True,False,False,True,False
4,True,True,True,False,False,True,False
5,False,True,True,False,False,False,True
6,True,False,False,False,False,False,True
7,True,False,False,True,False,True,False
8,True,True,False,True,False,False,False
9,True,False,False,True,False,False,True


In [29]:
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.7,(Butter)
1,0.4,(Cheese)
2,0.6,(Onion)
3,0.4,(Sugar)
4,0.5,(Water)
5,0.4,"(Onion, Butter)"


In [30]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Onion),(Butter),0.6,0.7,0.4,0.666667,0.952381,-0.02,0.900000
1,(Butter),(Onion),0.7,0.6,0.4,0.571429,0.952381,-0.02,0.933333


In [45]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Onion),(Butter),0.6,0.7,0.4,0.666667,0.952381,-0.02,0.900000
1,(Butter),(Onion),0.7,0.6,0.4,0.571429,0.952381,-0.02,0.933333


In [37]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(Onion),(Butter),0.6,0.7,0.4,0.666667,0.952381,-0.02,0.900000,1
1,(Butter),(Onion),0.7,0.6,0.4,0.571429,0.952381,-0.02,0.933333,1


In [41]:
rules[ (rules['antecedent_len'] >= 0) &
       (rules['confidence'] > 0.5) &
       (rules['lift'] > .9) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(Onion),(Butter),0.6,0.7,0.4,0.666667,0.952381,-0.02,0.900000,1
1,(Butter),(Onion),0.7,0.6,0.4,0.571429,0.952381,-0.02,0.933333,1


In [42]:
#select the ones you want
rules[rules['antecedents'] == {'Onion'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(Onion),(Butter),0.6,0.7,0.4,0.666667,0.952381,-0.02,0.9,1
